# railファイル確認用

In [4]:
import boto3
import os
import importlib
import shelve
import copy
import streamlit as st
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
import pandas as pd
import numpy as np
import datetime
# import src.utilsST_01 as utlst
import src.helpers as helpers
import src.visualize as vis
from src.config import appProperties
from src.trolley import trolley
from src.similar_pixel import pixel
from pprint import pprint
importlib.reload(helpers)
importlib.reload(vis)

config = appProperties('config.yml')

2023-07-07 01:57:23.391 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
2023-07-07 01:57:28.634 INFO    matplotlib.font_manager: generated new fontManager


In [5]:
images_path = helpers.list_imagespath_nonCache(config.image_dir)
# dir_area = "Chuo_01_Tokyo-St_up_20230201_knight"
# dir_area = "Chuo_02_Tokyo-Kanda_up_20230201_knight"
# dir_area = "Kawagoe_02_Nishioomiya-Sashiougi_down_20220626_knight"
# dir_area = "Chuo_24_Higashinakano-Nakano_down_20230201_day"
dir_area = "Takasaki_01_Omiya-Miyahara_up_20230217_day"
camera_num = "HD11"
target_dir = config.image_dir + "/" + dir_area + "/" + camera_num
outpath = config.output_dir + "/" + dir_area + "/" + camera_num

base_images = helpers.list_images(target_dir)

rail_fpath = outpath + "/rail.shelve"

# with shelve.open(rail_fpath) as rail:
#     rail_dict = copy.deepcopy(rail)

# rail = shelve.open(rail_fpath)

# shelveファイルの中身を1階層ずつチェックする

In [6]:
with shelve.open(rail_fpath) as rail:
    rail_dict = copy.deepcopy(rail)

keys = rail_dict.keys()
for i, key in enumerate(keys):
    print(f'{i}: {key}')

0: name
1: HD11


In [4]:
keys = rail_dict[camera_num].keys()
for i, key in enumerate(keys):
    print(f'{i}: {key}')

0: imgs/Takasaki_01_Omiya-Miyahara_up_20230217_day/HD11/2022_0599_HD11_01_00012454.jpg
1: imgs/Takasaki_01_Omiya-Miyahara_up_20230217_day/HD11/2022_0599_HD11_01_00012455.jpg
2: imgs/Takasaki_01_Omiya-Miyahara_up_20230217_day/HD11/2022_0599_HD11_01_00012456.jpg
3: imgs/Takasaki_01_Omiya-Miyahara_up_20230217_day/HD11/2022_0599_HD11_01_00012457.jpg
4: imgs/Takasaki_01_Omiya-Miyahara_up_20230217_day/HD11/2022_0599_HD11_01_00012458.jpg
5: imgs/Takasaki_01_Omiya-Miyahara_up_20230217_day/HD11/2022_0599_HD11_01_00012459.jpg
6: imgs/Takasaki_01_Omiya-Miyahara_up_20230217_day/HD11/2022_0599_HD11_01_00012460.jpg
7: imgs/Takasaki_01_Omiya-Miyahara_up_20230217_day/HD11/2022_0599_HD11_01_00012461.jpg
8: imgs/Takasaki_01_Omiya-Miyahara_up_20230217_day/HD11/2022_0599_HD11_01_00012462.jpg
9: imgs/Takasaki_01_Omiya-Miyahara_up_20230217_day/HD11/2022_0599_HD11_01_00012463.jpg
10: imgs/Takasaki_01_Omiya-Miyahara_up_20230217_day/HD11/2022_0599_HD11_01_00012464.jpg
11: imgs/Takasaki_01_Omiya-Miyahara_up_202

In [7]:
image_path = base_images[0]
image_path

'imgs/Takasaki_01_Omiya-Miyahara_up_20230217_day/HD11/2022_0599_HD11_01_00012454.jpg'

In [8]:
keys = rail_dict[camera_num][image_path].keys()
for i, key in enumerate(keys):
    print(f'{i}: {key}')

0: trolley1


In [9]:
with shelve.open(rail_fpath) as rail:
    trolley_dict = copy.deepcopy(rail[camera_num][image_path])

In [10]:
len(trolley_dict)

1

In [11]:
for trolley_id in config.trolley_ids[:len(trolley_dict)]:
    print(trolley_id)
    for key in trolley_dict[trolley_id].keys():
        print(key)

trolley1
estimated_upper_edge
estimated_lower_edge
estimated_width
estimated_slope
brightness_center
brightness_mean
brightness_std
estimated_upper_edge_variance
estimated_lower_edge_variance
estimated_slope_variance
trolley_end_reason
mask_edgelog_1
mask_edgelog_2


In [10]:
trolley_dict[config.trolley_ids[0]]["estimated_upper_edge"]

[961,
 961,
 961,
 961,
 961,
 961,
 961,
 962,
 962,
 962,
 961,
 962,
 962,
 962,
 962,
 962,
 962,
 962,
 962,
 962,
 962,
 962,
 962,
 962,
 962,
 962,
 962,
 962,
 962,
 962,
 962,
 962,
 962,
 962,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 963,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 964,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 965,
 966,
 966,
 966,
 966,
 966,
 966,
 966,
 966,
 966,
 966,
 966,
 966,
 966,
 966

In [ ]:
keys = rail_dict[camera_num][image_path]["trolley1"].keys()
for i, key in enumerate(keys):
    # print(f'{i}: {key}')
    try:
        key_len = len(rail_dict[camera_num][image_path]["trolley1"][key])
    except:
        key_len = 0
    print(f'{key} > length:{key_len}')

In [ ]:
image_path = base_images[0]
rail["HD11"][image_path]["trolley1"]["estimated_upper_edge"]

In [ ]:
'''
case1: xのみnp.array   -> 2023-05-24 00:44:31.574638 Process end :0:00:45.240699
case2: x,yをnp.array   -> 2023-05-24 02:24:05.555617 Process end :0:00:57.180313
case3: rail_dict化     -> 2023-05-24 04:58:30.819891 Process end :0:00:07.624567
case4: shelve.open追加 -> 2023-05-24 05:17:12.809013 Process end :0:00:03.504746
'''

dt01 = datetime.datetime.now()

img_num = 50

with shelve.open(rail_fpath) as rail:
    trolley_dict = copy.deepcopy(rail[camera_num])

for idx, image_path in enumerate(base_images):
    print(f"{idx}> {image_path}")
    x_values = np.array([n + 1000 * idx for n in trolley_dict[image_path][config.trolley_ids[0]]["ix"]])
    for trolley_id in config.trolley_ids:
        plt.plot(x_values, trolley_dict[image_path][trolley_id]["estimated_upper_edge"])
        plt.plot(x_values, trolley_dict[image_path][trolley_id]["estimated_lower_edge"])
    if idx >= img_num-1:
        print("break")
        break

plt.show()

dt02 = datetime.datetime.now()
prc_time = dt02 - dt01
print(str(datetime.datetime.now()) + f' Process end :{prc_time}')

In [ ]:
for key in trolley_dict[base_images[0]]['trolley1'].keys():
    print(key)

In [ ]:
print("最後の検出結果")
for trolley_id in config.trolley_ids:
    print(f"trolley_id:{trolley_id}")
    print(f'estimated_upper_edge:{trolley_dict[image_path][trolley_id]["estimated_upper_edge"][999]}')
    print(f'estimated_lower_edge:{trolley_dict[image_path][trolley_id]["estimated_lower_edge"][999]}')
    print(f'brightness_std:{trolley_dict[image_path][trolley_id]["brightness_std"][999]}')
    print(f'brightness_center:{trolley_dict[image_path][trolley_id]["brightness_center"][999]}')

# 結果画像を表示する

In [ ]:
with shelve.open(rail_fpath) as rail:
    trolley_dict = copy.deepcopy(rail[camera_num])

idx = 0
image_path = base_images[idx]
img = Image.open(image_path)
print(f"{idx}> {image_path}")

x_values = np.array([n + 1000 * idx for n in trolley_dict[image_path][config.trolley_ids[0]]["ix"]])
for trolley_id in config.trolley_ids:
    plt.plot(x_values, trolley_dict[image_path][trolley_id]["estimated_upper_edge"])
    plt.plot(x_values, trolley_dict[image_path][trolley_id]["estimated_lower_edge"])


In [ ]:
# 画像とデータの読み込み
with shelve.open(rail_fpath) as rail:
    trolley_dict = copy.deepcopy(rail[camera_num])

idx = 0
image_path = base_images[idx]
img = Image.open(image_path)
print(f"{idx}> {image_path}")

# 画像をnumpy配列に変換
img_array = np.array(img)

# ランダムに1000画素を選択し、その平均輝度を背景の輝度とする
random_pixels = img_array[np.random.randint(0, img_array.shape[0], 1000), np.random.randint(0, img_array.shape[1], 1000)]
background_brightness = random_pixels.mean()

# データを描画
x_values = trolley_dict[image_path][config.trolley_ids[0]]["ix"]
for trolley_id in config.trolley_ids:
    upper_edge = trolley_dict[image_path][trolley_id]["estimated_upper_edge"]
    lower_edge = trolley_dict[image_path][trolley_id]["estimated_lower_edge"]
    for x, y1, y2 in zip(x_values, upper_edge, lower_edge):
        # estimated_upper_edgeとestimated_lower_edgeが0でない場合のみ色を変更
        if y1 != 0:
            color_upper = [0, 255, 0] if background_brightness < 128 else [255, 0, 0]  # 緑または赤
            img_array[y1, x] = color_upper
        if y2 != 0:
            color_lower = [0, 255, 0] if background_brightness < 128 else [255, 0, 0]  # 緑または赤
            img_array[y2, x] = color_lower

# 変更後の画像を表示
display_img = Image.fromarray(img_array)
display_img

# 結果をCSV出力する

In [ ]:
# CSVファイルの保存パスを指定
csv_fpath = rail_fpath.replace(".shelve", ".csv")

# shelveファイルを読み込む
with shelve.open(rail_fpath) as rail:
    trolley_dict = copy.deepcopy(rail[camera_num])

In [ ]:
base_images = list(trolley_dict.keys())
print(f"Images count: {len(base_images)}")
img_path = base_images[int(input("set index number: "))]
print(f"img_path: {img_path}")

In [ ]:
config.trolley_ids

In [ ]:
print(f"img_path: {trolley_dict[img_path]}")
for key in trolley_dict[img_path].keys():
    print(key)

In [ ]:
for key in trolley_dict[img_path][config.trolley_ids[0]].keys():
    print(key)

In [ ]:
trolley_id = config.trolley_ids[0]
key = "estimated_upper_edge"
print(trolley_id)
print(type(trolley_dict[img_path][trolley_id][key]))
print(len(trolley_dict[img_path][trolley_id][key]))
print(trolley_dict[img_path][trolley_id][key])

In [33]:
# 単体テスト用
# shelveをCSVファイルとして書き出す

# CSVファイルの保存パスを指定
csv_fpath = rail_fpath.replace(".shelve", ".csv")
print(f"csv_fpath:{csv_fpath}")

# 1画像あたりの間引く数を指定
thin_out = 50

# shelveファイルを読み込む
with shelve.open(rail_fpath) as rail:
    trolley_dict = copy.deepcopy(rail[camera_num])

csv_fpath:output/Takasaki_01_Omiya-Miyahara_up_20230217_day/HD11/rail.csv


In [35]:
img_path = 'imgs/Takasaki_01_Omiya-Miyahara_up_20230217_day/HD11/2022_0599_HD11_01_00012454.jpg'
trolley_dict[img_path].keys()

dict_keys(['trolley1'])

In [36]:
trolley_id = config.trolley_ids[0]
column_name = [trolley_id + "_" + key for key in list(trolley_dict[img_path][trolley_id].keys())]
column_name

['trolley1_estimated_upper_edge',
 'trolley1_estimated_lower_edge',
 'trolley1_estimated_width',
 'trolley1_estimated_slope',
 'trolley1_brightness_center',
 'trolley1_brightness_mean',
 'trolley1_brightness_std',
 'trolley1_estimated_upper_edge_variance',
 'trolley1_estimated_lower_edge_variance',
 'trolley1_estimated_slope_variance',
 'trolley1_trolley_end_reason',
 'trolley1_mask_edgelog_1',
 'trolley1_mask_edgelog_2']

In [43]:
column_name = ["ix"]
for trolley_id in config.trolley_ids:
    trolley_column = [trolley_id + "_" + key for key in config.result_keys]
    column_name = column_name + trolley_column
print(column_name)

['ix', 'trolley1_estimated_upper_edge', 'trolley1_estimated_lower_edge', 'trolley1_estimated_width', 'trolley1_brightness_center', 'trolley1_brightness_mean', 'trolley1_brightness_std', 'trolley1_estimated_slope', 'trolley1_upper_boundary', 'trolley1_lower_boundary', 'trolley1_estimated_upper_edge_variance', 'trolley1_estimated_lower_edge_variance', 'trolley1_estimated_slope_variance', 'trolley1_trolley_end_reason', 'trolley1_mask_edgelog_1', 'trolley1_mask_edgelog_2', 'trolley2_estimated_upper_edge', 'trolley2_estimated_lower_edge', 'trolley2_estimated_width', 'trolley2_brightness_center', 'trolley2_brightness_mean', 'trolley2_brightness_std', 'trolley2_estimated_slope', 'trolley2_upper_boundary', 'trolley2_lower_boundary', 'trolley2_estimated_upper_edge_variance', 'trolley2_estimated_lower_edge_variance', 'trolley2_estimated_slope_variance', 'trolley2_trolley_end_reason', 'trolley2_mask_edgelog_1', 'trolley2_mask_edgelog_2', 'trolley3_estimated_upper_edge', 'trolley3_estimated_lower_

In [44]:
column_name = [
    f"{trolley_id}_{key}"
    for trolley_id in config.trolley_ids
    for key in config.result_keys
]
print(column_name)

['trolley1_estimated_upper_edge', 'trolley1_estimated_lower_edge', 'trolley1_estimated_width', 'trolley1_brightness_center', 'trolley1_brightness_mean', 'trolley1_brightness_std', 'trolley1_estimated_slope', 'trolley1_upper_boundary', 'trolley1_lower_boundary', 'trolley1_estimated_upper_edge_variance', 'trolley1_estimated_lower_edge_variance', 'trolley1_estimated_slope_variance', 'trolley1_trolley_end_reason', 'trolley1_mask_edgelog_1', 'trolley1_mask_edgelog_2', 'trolley2_estimated_upper_edge', 'trolley2_estimated_lower_edge', 'trolley2_estimated_width', 'trolley2_brightness_center', 'trolley2_brightness_mean', 'trolley2_brightness_std', 'trolley2_estimated_slope', 'trolley2_upper_boundary', 'trolley2_lower_boundary', 'trolley2_estimated_upper_edge_variance', 'trolley2_estimated_lower_edge_variance', 'trolley2_estimated_slope_variance', 'trolley2_trolley_end_reason', 'trolley2_mask_edgelog_1', 'trolley2_mask_edgelog_2', 'trolley3_estimated_upper_edge', 'trolley3_estimated_lower_edge',

In [46]:
df = pd.DataFrame(np.nan, index=np.arange(config.max_len), columns=column_name)
df

,trolley1_estimated_upper_edge,trolley1_estimated_lower_edge,trolley1_estimated_width,trolley1_brightness_center,trolley1_brightness_mean,trolley1_brightness_std,trolley1_estimated_slope,trolley1_upper_boundary,trolley1_lower_boundary,trolley1_estimated_upper_edge_variance,...,trolley3_brightness_std,trolley3_estimated_slope,trolley3_upper_boundary,trolley3_lower_boundary,trolley3_estimated_upper_edge_variance,trolley3_estimated_lower_edge_variance,trolley3_estimated_slope_variance,trolley3_trolley_end_reason,trolley3_mask_edgelog_1,trolley3_mask_edgelog_2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
trolley_id = config.trolley_ids[0]
helpers.trim_trolley_dict(config, trolley_dict, img_path)
df = pd.DataFrame(trolley_dict[img_path][trolley_id]).copy()

In [54]:
df

,estimated_upper_edge,estimated_lower_edge,estimated_width,estimated_slope,brightness_center,brightness_mean,brightness_std,estimated_upper_edge_variance,estimated_lower_edge_variance,estimated_slope_variance,trolley_end_reason,mask_edgelog_1,mask_edgelog_2
0,960,980,22,-0.000250,255.0,200.625,78.37500,0.750488,0.750488,0.001049,NaN,0,0
1,960,979,21,-0.000609,255.0,209.125,70.62500,0.602051,0.602051,0.001098,NaN,0,0
2,960,979,21,-0.001028,255.0,208.625,71.06250,0.504395,0.503906,0.001144,NaN,0,0
3,961,979,20,-0.001484,255.0,215.750,65.50000,0.435547,0.435547,0.001188,NaN,0,0
4,961,979,20,-0.001963,255.0,216.375,64.50000,0.385498,0.385254,0.001229,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,985,1001,18,0.016609,255.0,232.500,43.56250,0.158936,0.158936,0.000931,NaN,0,0
996,985,1001,18,0.016877,255.0,232.250,44.09375,0.158936,0.158936,0.000931,NaN,0,0
997,985,1001,18,0.017024,255.0,235.875,38.06250,0.158936,0.158936,0.000931,NaN,0,0
998,985,1001,18,0.019799,255.0,233.500,42.53125,0.158936,0.158936,0.000931,NaN,0,0


In [61]:
for i, key in enumerate(config.result_keys):
    print(f'{i}> key:{key} ->{bool(key in df.columns)}')

0> key:estimated_upper_edge ->True
1> key:estimated_lower_edge ->True
2> key:estimated_width ->True
3> key:brightness_center ->True
4> key:brightness_mean ->True
5> key:brightness_std ->True
6> key:estimated_slope ->True
7> key:upper_boundary ->False
8> key:lower_boundary ->False
9> key:estimated_upper_edge_variance ->True
10> key:estimated_lower_edge_variance ->True
11> key:estimated_slope_variance ->True
12> key:trolley_end_reason ->True
13> key:mask_edgelog_1 ->True
14> key:mask_edgelog_2 ->True


In [59]:
len(config.result_keys)

15

In [60]:
len(df.columns)

13

In [113]:
trolley_id = config.trolley_ids[0]
df1 = pd.DataFrame(trolley_dict[img_path][trolley_id])
diff_list = list(set(config.result_keys) - set(list(df1.columns)))
for column in diff_list:
    df1[column] = pd.Series([np.nan]*config.max_len, index=df.index)
df1 = df1.reindex(columns=config.result_keys, copy=False)
df1.columns

Index(['estimated_upper_edge', 'estimated_lower_edge', 'estimated_width',
       'brightness_center', 'brightness_mean', 'brightness_std',
       'estimated_slope', 'upper_boundary', 'lower_boundary',
       'estimated_upper_edge_variance', 'estimated_lower_edge_variance',
       'estimated_slope_variance', 'trolley_end_reason', 'mask_edgelog_1',
       'mask_edgelog_2'],
      dtype='object')

In [114]:
trolley_id = config.trolley_ids[0]
df2 = pd.DataFrame(trolley_dict[img_path][trolley_id])
diff_list = list(set(config.result_keys) - set(list(df2.columns)))
for column in diff_list:
    df2[column] = pd.Series([np.nan]*config.max_len, index=df.index)
df2 = df2.reindex(columns=config.result_keys, copy=False)
df2.columns

Index(['estimated_upper_edge', 'estimated_lower_edge', 'estimated_width',
       'brightness_center', 'brightness_mean', 'brightness_std',
       'estimated_slope', 'upper_boundary', 'lower_boundary',
       'estimated_upper_edge_variance', 'estimated_lower_edge_variance',
       'estimated_slope_variance', 'trolley_end_reason', 'mask_edgelog_1',
       'mask_edgelog_2'],
      dtype='object')

In [115]:
trolley_id = config.trolley_ids[0]
df3 = pd.DataFrame(trolley_dict[img_path][trolley_id])
diff_list = list(set(config.result_keys) - set(list(df3.columns)))
for column in diff_list:
    df3[column] = pd.Series([np.nan]*config.max_len, index=df.index)
df3 = df3.reindex(columns=config.result_keys, copy=False)
df3.columns

Index(['estimated_upper_edge', 'estimated_lower_edge', 'estimated_width',
       'brightness_center', 'brightness_mean', 'brightness_std',
       'estimated_slope', 'upper_boundary', 'lower_boundary',
       'estimated_upper_edge_variance', 'estimated_lower_edge_variance',
       'estimated_slope_variance', 'trolley_end_reason', 'mask_edgelog_1',
       'mask_edgelog_2'],
      dtype='object')

In [116]:
df_trolley = pd.DataFrame()
df_trolley = pd.concat([df_trolley, df1], axis=1)
df_trolley = pd.concat([df_trolley, df2], axis=1)
df_trolley = pd.concat([df_trolley, df3], axis=1)
df_trolley.columns

Index(['estimated_upper_edge', 'estimated_lower_edge', 'estimated_width',
       'brightness_center', 'brightness_mean', 'brightness_std',
       'estimated_slope', 'upper_boundary', 'lower_boundary',
       'estimated_upper_edge_variance', 'estimated_lower_edge_variance',
       'estimated_slope_variance', 'trolley_end_reason', 'mask_edgelog_1',
       'mask_edgelog_2', 'estimated_upper_edge', 'estimated_lower_edge',
       'estimated_width', 'brightness_center', 'brightness_mean',
       'brightness_std', 'estimated_slope', 'upper_boundary', 'lower_boundary',
       'estimated_upper_edge_variance', 'estimated_lower_edge_variance',
       'estimated_slope_variance', 'trolley_end_reason', 'mask_edgelog_1',
       'mask_edgelog_2', 'estimated_upper_edge', 'estimated_lower_edge',
       'estimated_width', 'brightness_center', 'brightness_mean',
       'brightness_std', 'estimated_slope', 'upper_boundary', 'lower_boundary',
       'estimated_upper_edge_variance', 'estimated_lower_edge_var

In [119]:
df_trolley.columns = column_name
df_trolley.columns

Index(['trolley1_estimated_upper_edge', 'trolley1_estimated_lower_edge',
       'trolley1_estimated_width', 'trolley1_brightness_center',
       'trolley1_brightness_mean', 'trolley1_brightness_std',
       'trolley1_estimated_slope', 'trolley1_upper_boundary',
       'trolley1_lower_boundary', 'trolley1_estimated_upper_edge_variance',
       'trolley1_estimated_lower_edge_variance',
       'trolley1_estimated_slope_variance', 'trolley1_trolley_end_reason',
       'trolley1_mask_edgelog_1', 'trolley1_mask_edgelog_2',
       'trolley2_estimated_upper_edge', 'trolley2_estimated_lower_edge',
       'trolley2_estimated_width', 'trolley2_brightness_center',
       'trolley2_brightness_mean', 'trolley2_brightness_std',
       'trolley2_estimated_slope', 'trolley2_upper_boundary',
       'trolley2_lower_boundary', 'trolley2_estimated_upper_edge_variance',
       'trolley2_estimated_lower_edge_variance',
       'trolley2_estimated_slope_variance', 'trolley2_trolley_end_reason',
       'trolley

In [117]:
len(df_trolley.columns)

45

In [120]:
len(df_trolley.columns)

45

In [118]:
len(column_name)

45

In [ ]:
dfs = pd.DataFrame()
for img_idx, img_path in enumerate(base_images):
    # print(f"{img_idx}> img_path: {img_path}")
    if len(trolley_dict[img_path]):
        # img_pathの結果が空でないときに実行する
        df_trolley = helpers.read_trolley_dict(config, trolley_dict, img_idx, img_path, thin_out).copy()
        # ixの値が連番になるように修正
        df_trolley['ix'] = df_trolley['ix'] + 1000 * img_idx
    elif img_idx:
        # img_pathの結果が空のとき & img_idxが0以外のとき
        # print("trolley_dict[img_path] is empty")
        df_trolley = helpers.trolley_dict_fillna(img_idx, img_path, dfs.columns).copy()
    elif img_idx:
        # 1枚目の画像でエラーになる
        print("解析結果がないため、CSVファイルを生成できませんでした。")
        break
    dfs = pd.concat([dfs, df_trolley], ignore_index=True)

# estimated_widthの標準偏差を計算して追記する
dfs = helpers.width_std_calc(config, dfs, thin_out)
# データを間引く
dfs = dfs[::thin_out]
print("convert shelve to dataframe")
dfs

In [ ]:
dfs.columns

In [ ]:
# CSVファイルに変換する
dfs.to_csv(csv_fpath, encoding='cp932')

# CSVファイルからグラフを表示する

In [ ]:
from bokeh.plotting import figure, gridplot, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import d3
output_notebook()


def plot_fig_bokeh(csv_fpath, graph_height, graph_width):

    # CSVファイルをデータフレームとして読み込む
    df_csv = pd.read_csv(csv_fpath, encoding='cp932')
    
    # CSVから作成したデータフレームをbokeh形式で読み込む
    source = ColumnDataSource(data=df_csv)
    
    # グラフの色情報を設定する
    # グラフの数に合わせて12色だけ取得する
    colors = d3["Category20"][12]

    # ツールチップを設定
    TOOLTIPS_EDGE=[
        ('index', '@img_idx'),
        ('image_name', '@image_name'),
        ('upper_edge1', '@trolley1_estimated_upper_edge'),
        ('lower_edge1', '@trolley1_estimated_lower_edge'),
        ('upper_edge2', '@trolley2_estimated_upper_edge'),
        ('lower_edge2', '@trolley2_estimated_lower_edge'),
        ('upper_edge3', '@trolley3_estimated_upper_edge'),
        ('lower_edge3', '@trolley3_estimated_lower_edge'),
    ]
    TOOLTIPS_WIDTH=[
        ('index', '@img_idx'),
        ('image_name', '@image_name'),
        ('estimated_width1', '@trolley1_estimated_width'),
        ('estimated_width2', '@trolley2_estimated_width'),
        ('estimated_width3', '@trolley3_estimated_width'),
    ]
    TOOLTIPS_BRIGHTNESS=[
        ('index', '@img_idx'),
        ('image_name', '@image_name'),
        ('brightness_center1', '@trolley1_brightness_center'),
        ('brightness_center2', '@trolley2_brightness_center'),
        ('brightness_center3', '@trolley3_brightness_center'),
    ]

    # グラフを作成
    p_edge = figure(
        title="Upper and Lower Edge",
        sizing_mode="stretch_width",
        tooltips=TOOLTIPS_EDGE,
        height=int(graph_height),
        width=int(graph_width)
    )
    p_width = figure(
        title="Width",
        sizing_mode="stretch_width",
        tooltips=TOOLTIPS_WIDTH,
        x_range=p_edge.x_range,
        height=int(graph_height),
        width=int(graph_width)
    )
    p_center = figure(
        title="Brightness Center",
        sizing_mode="stretch_width",
        tooltips=TOOLTIPS_BRIGHTNESS,
        x_range=p_edge.x_range,
        height=int(graph_height),
        width=int(graph_width)
    )

    # グラフを表示する領域を作成
    grid = gridplot(
        [[p_edge], [p_width], [p_center]],
        toolbar_location="above"
    )

    # データを追加
    x_values = "ix"
    upper_edge1 = "trolley1_estimated_upper_edge"
    lower_edge1 = "trolley1_estimated_lower_edge"
    upper_edge2 = "trolley2_estimated_upper_edge"
    lower_edge2 = "trolley2_estimated_lower_edge"
    upper_edge3 = "trolley3_estimated_upper_edge"
    lower_edge3 = "trolley3_estimated_lower_edge"
    estimated_width1 = "trolley1_estimated_width"
    estimated_width2 = "trolley2_estimated_width"
    estimated_width3 = "trolley3_estimated_width"
    brightness_center1 = "trolley1_brightness_center"
    brightness_center2 = "trolley2_brightness_center"
    brightness_center3 = "trolley3_brightness_center"

    # グラフにデータを追加
    # 1つ目のグラフ（Upper and Lower Edge）
    p_edge.line(x_values, upper_edge1, line_color=colors[0], source=source)
    p_edge.line(x_values, upper_edge2, line_color=colors[1], source=source)
    p_edge.line(x_values, upper_edge3, line_color=colors[2], source=source)
    p_edge.line(x_values, lower_edge1, line_color=colors[3], source=source)
    p_edge.line(x_values, lower_edge2, line_color=colors[4], source=source)
    p_edge.line(x_values, lower_edge3, line_color=colors[5], source=source)

    # 2つ目のグラフ（Brightness Std）
    p_width.line(x_values, estimated_width1, line_color=colors[6], source=source)
    p_width.line(x_values, estimated_width2, line_color=colors[7], source=source)
    p_width.line(x_values, estimated_width3, line_color=colors[8], source=source)

    # 3つ目のグラフ（Brightness Center）
    p_center.line(x_values, brightness_center1, line_color=colors[9], source=source)
    p_center.line(x_values, brightness_center2, line_color=colors[10], source=source)
    p_center.line(x_values, brightness_center3, line_color=colors[11], source=source)

    return grid

# メインループ
# CSVファイルの保存パスを指定
csv_fpath = rail_fpath.replace(".shelve", ".csv")
print(f"csv path:{csv_fpath}")

graph_height = 200
graph_width = 1000

grid = plot_fig_bokeh(csv_fpath, graph_height, graph_width)

show(grid)


In [ ]:
# グラフを作成
p_edge = figure(
    title="Upper and Lower Edge",
    sizing_mode="stretch_width",
    height=int(graph_height)
)
p_width = figure(
    title="Width",
    sizing_mode="stretch_width",
    x_range=p_edge.x_range,
    height=int(graph_height)
)
p_center = figure(
    title="Brightness Center",
    sizing_mode="stretch_width",
    x_range=p_edge.x_range,
    height=int(graph_height)
)

# グラフを表示する領域を作成
grid = gridplot(
    [[p_edge], [p_width], [p_center]],
    toolbar_location="above"
)

# データを追加
x_values = df_csv["ix"]
upper_edge = df_csv["trolley1_estimated_upper_edge"]
lower_edge = df_csv["trolley1_estimated_lower_edge"]
estimated_width = df_csv["trolley1_estimated_width"]
brightness_center = df_csv["trolley1_brightness_center"]

# グラフにデータを追加
# 1つ目のグラフ（Upper and Lower Edge）
p_edge.line(x_values, upper_edge, line_color="blue")
p_edge.line(x_values, lower_edge, line_color="red")

# 2つ目のグラフ（Brightness Std）
p_width.line(x_values, estimated_width, line_color="green")

# 3つ目のグラフ（Brightness Center）
p_center.line(x_values, brightness_center, line_color="orange")

show(grid)

In [130]:
config.camera_name_to_type

{'高(左)': 'HD11',
 '高(右)': 'HD12',
 '中(左)': 'HD21',
 '中(右)': 'HD22',
 '低(左)': 'HD31',
 '低(右)': 'HD32'}

In [ ]:
def set_column_names(config, trolley_dict):
    column_names = []
    for idx, trolley_id in enumerate(config.trolley_ids):
        for key in trolley_dict[img_path][trolley_id].keys():
            column_names.append(trolley_id + "_" + key)
    return column_names

column_names = set_column_names(config, trolley_dict)
pprint(f"column length: {len(column_names)}")
pprint(column_names)

In [ ]:
print(f"img_path: {img_path}")
for key in trolley_dict[img_path][config.trolley_ids[0]].keys():
    pprint(f"{key} > {len(trolley_dict[img_path][config.trolley_ids[0]][key])}")